# 📦 Prétraitement EEG – Multi-sujets avec structure réutilisable

In [1]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 📁 Répertoire contenant les fichiers EDF
DATA_DIR = r"C:\Users\Antoi\Documents\EEG-BCI\data\raw\physionet"
BAD_CHANNELS = ['FCZ', 'CZ', 'CPZ', 'FP1', 'FPZ', 'FP2', 'AFZ', 'FZ', 'PZ', 'POZ', 'OZ', 'IZ']
EVENT_ID = dict(rest=1, left=2, right=3)  # À ajuster selon les annotations exactes

In [3]:
X_all, y_all = [], []
edf_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.edf')]

for file in edf_files:
    print(f"🔄 Traitement de {file}")
    raw = mne.io.read_raw_edf(os.path.join(DATA_DIR, file), preload=True)
    raw.set_eeg_reference('average', projection=True)
    montage = mne.channels.make_standard_montage("standard_1020")
    raw.set_montage(montage, on_missing="ignore")

    # ⚠️ Supprimer les canaux problématiques
    raw.pick_channels([ch for ch in raw.ch_names if ch not in BAD_CHANNELS])

    # 🎚️ Filtrage EEG classique
    raw.filter(1., 40., fir_design='firwin')

    # ⏱️ Extraction des événements
    events, _ = mne.events_from_annotations(raw)
    epochs = mne.Epochs(raw, events, event_id=EVENT_ID, tmin=0, tmax=2,
                        baseline=None, preload=True, detrend=1)

    # 🧮 Normalisation Z-score
    data = epochs.get_data()
    data = (data - data.mean(axis=-1, keepdims=True)) / data.std(axis=-1, keepdims=True)

    # 📐 Reshape pour EEGNet : [N, 1, C, T]
    X = data[:, np.newaxis, :, :]
    y = epochs.events[:, 2]

    X_all.append(X)
    y_all.append(y)

🔄 Traitement de S001R06.edf
Extracting EDF parameters from C:\Users\Antoi\Documents\EEG-BCI\data\raw\physionet\S001R06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passba

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


🔄 Traitement de S001R10.edf
Extracting EDF parameters from C:\Users\Antoi\Documents\EEG-BCI\data\raw\physionet\S001R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passba

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'),

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.0s finished


In [4]:
# 💾 Sauvegarde finale
X_all = np.concatenate(X_all, axis=0)
y_all = np.concatenate(y_all, axis=0)
np.save("X_windows.npy", X_all)
np.save("y_windows.npy", y_all)
print(f"✅ Données totales : X = {X_all.shape}, y = {y_all.shape}")

✅ Données totales : X = (90, 1, 64, 321), y = (90,)
